In [ ]:
# ================================================== [ setting ] ==================================================
# !pip install konlpy
# !pip install pyLDAvis==2.1.2
import pandas as pd 
import re
from konlpy.tag import Komoran
from gensim import corpora
import gensim
import pyLDAvis.gensim
import warnings
warnings.filterwarnings(action = 'ignore')

policy_info = pd.read_csv("policy_info.csv", encoding = "utf-8")

In [ ]:
# ============================================== [ LDA modeling ] ======================================================
def LDA_modeling(df, num_topics = 10) :
  df = df.str.replace("\([^)]*\)", "") # 괄호안의 내용 제거 ex) 코로나(COVID-19) -> 코로나
  df = df.str.replace("\n\n", "") # 문단 구분자 제거 
  df = df.str.replace("[^a-zA-Z가-힣 ]", "") # 영어 / 한글 제외하고 모두 제거 
  
  processed_data = []
  for sentence in df : 
    komoran = Komoran()
    tag = komoran.pos(sentence) # 형태소 처리 ["단어", "품사"]
    nouns = [s for s, t in tag if t in ['SL', 'NNG', 'NNP'] and len(s) > 1] # 명사 태그 추출, 한글자 이상만 추출 # nng : 명사 nnp :  고유대명사 sl : 외래어  
    processed_data.append(nouns) # token 형태로 list에 저장

  dictionary = corpora.Dictionary(processed_data) # {512, 1} {522, 2} ... 
  corpus = [dictionary.doc2bow(text) for text in processed_data]

  NUM_TOPICS = num_topics
  ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word = dictionary, passes = 15)
  topics = ldamodel.print_topics(num_words = 10)
  
  pyLDAvis.enable_notebook()
  vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary) # 그래프 출력
  
  return pyLDAvis.display(vis)

In [ ]:
LDA_modeling(policy_info["policy_briefing"], num_topics = 10) # LDA과련 토픽 1. 코로나 + 추경, 청년 정책, 2. 부동산 규제 정책, 3. 에너지 산업이나, 계수값이 미비함 . 
# topic_n = 4 (optim.) 수정할것. 